# （復習）型ヒント
*  Pythonの型ヒント（Type Hinting）は，関数の引数や戻り値，変数がどのような型（クラス）であるかを明示するためのしくみ
*  型ヒント自体はプログラムの動作に影響を与えないが，オブジェクトのクラス（データ型）の取り扱いミスが見つけやすくなり，コードの可読性や保守性を高める効果がある
*  また，Google Coaboratoryなどの開発環境における補完機能やエラーチェックが強化され，開発がスムーズになる  

---
```Python
class Engine:
    def __init__(self, horsepower: int) -> None:
        self.horsepower = horsepower

class Car:
    def __init__(self, brand: str, horsepower: int) -> None:
        self.brand = brand
        self.engine: Engine = Engine(horsepower)  # Engineクラスのインスタンス生成

car = Car('Toyota', 150)
print(car.engine.horsepower) # 馬力
```
---
 
*  上のコードは，`Car`クラスと`Engine`クラスの2つのクラスが定義されており，`Car`クラスのインスタンス（車）が`Engine`クラスのインスタンス（エンジン）を所有するという関係（コンポジション）を表している
*  具体的には，`Car`クラスのインスタンスを生成すると，コンストラクタ内で`Engine`クラスのインスタンスを生成し，インスタンス属性の属性値として定義している
*  `Engine`クラスのインスタンスは，`Car`クラスのインスタンス`car`のインスタンス属性であるので，`del car`で`car`を削除すると，その属性であるインスタンスも削除される
*  この例におけるクラス図を下図に示す

<img src="./fig/composition_car.png" width="170">  


# 例外クラスの定義と不適切な例外処理

## （復習）例外処理の書式
*  Pythonは，コード実行中に次に何をすべきかの判断ができないと「例外オブジェクト」を生成する
*  例外オブジェクトを生成する例外クラスには，「NameError」「TypeError」「ValueError」などがある
*  例外処理とは，例外オブジェクトを適切に処理し，例外が発生してもプログラムが中断されないようにする処理である

  
---
```Python
try:
    # 例外が発生する可能性のある処理
except 捕捉する例外の種類（例外クラス）:
    # tryブロック内で例外が発生したときの処理
else:
    # 例外が発生しなかったときの処理
finally:
    # 例外が発生したかどうかに関係なく実行される処理（必ず実行される）
```
---

## （復習）`raise`文で例外を発生させる
*  `raise`文を使うと，特定の例外を意図的に発生できる
*  `raise`文を適切に利用することで，例外の管理と処理を効果的に行うことができる

**書式:**
```Python
raise 例外クラス名('メッセージ')
```

In [ ]:
class User:
  def __init__(self, name: str) -> None:
    # ガード節
    if len(name) > 20:
      raise ValueError('ユーザ名が20文字を超えています。') 
    self.name = name

try:
    name = input('ユーザ名を入力 >> ')
    user = User(name)
except ValueError as e:
    print(f'エラー発生： {e}')

## （復習）ガード節
*  上のコードの`if`文のように，不正な値が与えられた場合に例外を発生させることで，クラスやメソッドの正しい使用を強制できる
*  つまり，システムの利用者が間違った使い方をしても，エラー（例外）を出すことなく適切な対応をとることができる
*  不正な値が渡されたら例外を発生させるようなメソッドの先頭の記述（上のコードの`if`文）をガード節と呼ぶ

## 例外クラスの定義
*  例外クラスは自作することもできる
*  新しく定義する例外クラスは，Exceptionクラスを継承したクラスとする必要がある
*  既存の（組み込みの）例外クラスも，すべてExceptionクラスを継承している
  
---
```Python
def 例外クラス名(Exception):
    ...
```
---

In [ ]:
class MyError(Exception):
    pass

def is_positive(n: int) -> None:
    if n < 0:
        raise MyError('値は負であってはなりません。')

try:
    number = int(input('正の数を入力 >> '))
    is_positive(number)
except MyError as e:
    print(f'エラー発生： {e}')
else:
    print('正常に処理できました。')
finally:
    print('処理を終了します。')

## すべての例外を無視する
*  例外処理は安定した動作をするシステムを開発するうえで重要な機能
*  ただし，間違った使い方（アンチパターン）をすると問題が生じる
*  代表的な間違った使い方として，「すべての例外を無視する」がある
*  すべての例外を無視するには，以下のコードのように，すべての例外クラスが継承するExceptionクラスを補足すればよい
  
---
```Python
while True:
    try:
        n = int(input('整数値を入力してください：'))
        print(n)
        break
    except Exception:
        print('整数値ではありません。入力し直してください。')
```
---
  
*  しかし，上のコードだと，どのような例外を補足しているのかがわからない
*  開発者が想定していない例外が起きている可能性もある
*  これは，システムの深刻な問題につながるかもしれない
*  したがって，補足する例外クラスは，下のコードのように具体的に記述する法が望ましい（というか具体的に記述すべき）
  
---
```Python
while True:
    try:
        n = int(input('整数値を入力してください：'))
        print(n)
        break
    except ValueError:
        print('整数値ではありません。入力し直してください。')
```
---

# フールプルーフ

## （復習）フールプルーフとは
*  フールプルーフ（foolproof）とは，システムの利用者が本来のシステムの仕様から外れた誤った操作をしても，異常が起きないようにするといった考え方（設計思想）
*  **典型的なフールプルーフの例**: ファイルを保存しないで終了しようとしたときに，ファイルの保存を確認するメッセージを表示するといったような，利用者の入力ミスや操作ミスをあらかじめ想定した設計にする
*  **身の回りのフールプルーフの例**:
>*  ドアが完全に閉じないと起動しない電子レンジ
>*  蓋を閉めないと回転しない洗濯機
>*  座らないと作動しない温水便座
>*  転倒すると消える電気ストーブ
*  ガード節やカプセル化におけるおける情報の隠ぺい（属性などのアクセス制限）も，フールプルーフの考え方に従っている

## フールプルーフに基づくコードの例
*  コンストラクタ`__init__`: インスタンス化のときに円の半径をプライベート属性`__radius`として設定
*  `@property`デコレータと`@radius.setter`デコレータ: 半径に対してゲッター（getter）とセッター（setter）を定義し，外部から半径にアクセスしたり，変更できるようにしている
*  半径の変更時にはガード節によって，半径が負の値であれば例外ValueErrorを発生させる

In [ ]:
class Circle:
    def __init__(self, radius: float) -> None:
        self.__radius = radius # 円の半径（プライベート属性）

    # 半径の取得 (getter)
    @property
    def radius(self) -> float:
        return self.__radius

    # 半径の設定 (setter): ガード節付き
    @radius.setter
    def radius(self, value: float) -> None:
        if value < 0:
            raise ValueError('半径は0以上である必要があります')
        self.__radius = value


circle = Circle(5)
print(f"半径: {circle.radius}")  # getterによる属性の読み取り

# 半径の変更 (setterの呼び出し)
circle.radius = 8
print(f"変更後の半径: {circle.radius}")

# エラーチェック
# circle.radius = -3  # エラー: ガード節による例外発生

# クラスの依存関係
*  クラスAがクラスBを使用しているとき，「クラスAはクラスBに依存している」という
*  クラス間の依存関係には，主に継承とコンポジションがある
*  継承は「is-a関係」，コンポジションは「has-a関係」と呼ばれる関係にある

## 継承による依存関係の例
*  クラス`B`はクラス`A`を継承している
*  このとき，クラス`B`はクラス`A`に依存している
*  また，クラス`A`は，`method_a`メソッドの中で文字列（`str`のオブジェクト）を使っているので，クラス`A`は`str`クラスに依存しているという依存関係がある

In [ ]:
class A:
    def method_a(self) -> None:
        print('クラスAのメソッド')

class B(A):
    def method_b(self) -> None:
        super().method_a() # クラスAのメソッドを呼び出して使用

## コンポジションによる依存関係の例
*  クラス`B`はクラス`A`をインスタンス属性として所有している（コンポジション）
*  このとき，クラス`B`はクラス`A`に依存している
*  また，上の例と同様にクラス`A`は`str`クラスに依存している

In [2]:
class A:
    def method_a(self) -> None:
        print('クラスAのメソッド')

class B:
    def __init__(self) -> None:
        self.obj = A() # クラスAのインスタンスを生成
    def method_b(self) -> None:
        self.obj.method_a() # クラスAのメソッドを呼び出して使用

## 依存関係と変更のしやすさ
*  上の2つの例のように，クラス`B`がクラス`A`に依存しているのであれば...
>*  クラスAが定義されてないと，クラスBは存在できない（定義できない）
>*  クラスAの中身が変更されると，クラスBにも影響する
*  逆に...
>*  クラスBが定義されてなくても，クラスAは存在できる
>*  クラスBにの中身が変更されても，クラスAには影響がない
*  変更しやすいシステム（ソフトウェア）を作るためには...
>*  依存関係はなるべく少ない方が望ましい
>*  変更されないもの，変更が少ないものに依存するのが望ましい
*  上の2つの例で，クラス`A`が`str`クラスに依存しているように，組み込みクラスへの依存はあらゆるクラスで発生する
*  しかし，組み込みクラスが変更されることは稀なので，特に問題はない

# 情報システム（ソフトウェア）の保守
*  情報システム（ソフトウェア）の保守とは，システム運用中に発生する問題や要求に対して，システムを維持・改善する活動のことを指す
*  保守は，システムがリリースされた後も，その性能を維持し，適切な機能を提供し続けるために必要なプロセス 

## 保守の主な種類
*  修正保守: 情報システムの不具合やバグを修正するための保守
*  適応保守: 情報システムが動作する環境の変化（OSやハードウェア，周辺システムの変更）に対応するための保守
*  予防保守: 情報システムが将来的に安定して稼働し続けるために，潜在的な問題を予防するための保守
*  完全保守: 新しい機能の追加や，既存機能の改善に関する保守

## 保守性
*  保守性とは，情報システムが運用される中で，修正，改良，及び変更がどれだけ容易に行えるかを指す概念
*  情報システムは，リリース後も利用者の要望や技術の進歩に対応するために継続的な修正や改良が求められる
*  保守性が高いシステムは，次のような作業が効率的に行える
>*  バグ修正: システムにバグが発生した際，修正にかかる時間や労力を最小限に抑えることができる
>*  機能追加・変更: 新機能を追加したり，既存機能を改良する際に，コードの構造や依存関係が整っていればスムーズに対応できる
>*  環境対応: システムが稼働する環境（ハードウェアやOSなど）が変わった場合に，その変化に対して柔軟に対応できる
*  保守性の高いコードは可読性が高く，誰にでも理解しやすい構造になっている
*  また，保守性は情報システムの品質を測る重要な指標の一つである
*  下図は，[JIS X 25010:2013（IEC25010：2011）](https://kikakurui.com/x2/X25010-2013-01.html)に基づく品質モデルである

<img src="./fig/software_quality.png" width="700">
  

## 保守性とオブジェクト指向
*  オブジェクト指向は，保守性を向上させるための重要なアプローチである
*  ここで，これまで学んできたオブジェクト指向の重要な概念と保守性との関連を整理しておく

### カプセル化と保守性
*  オブジェクト指向では，データとその操作を1つのオブジェクト（クラス）としてまとめるカプセル化が行われる
*  カプセル化によって，データ（属性）と操作（メソッド）が外部から直接操作されないように保護することができる
*  適切にカプセル化されていれば，クラス内部の実装を変更しても，そのクラスの外部に影響を与えにくくなるため，コードの変更や修正が他の部分に波及しにくくなり，保守性が向上する

### 継承と保守性
*  オブジェクト指向では，既存のクラスから新しいクラスを作る際に，スーパークラスの機能を引き継ぐ継承が使われる
*  継承によって，同じ機能を何度も記述せずに再利用でき，コードの重複を避けられる
*  共通の機能を継承元クラスにまとめておくことで，1箇所を修正すれば継承しているクラス全体に反映される
*  その結果，変更や修正時の効率が向上し，保守性が向上する

### ポリモーフィズムと保守性
*  オブジェクト指向では，異なるクラスが共通のインターフェースを持つことで，ポリモーフィズム（多態性）を実現する
*  これにより，同じメソッド名で異なる動作（実装）を行うことが可能となる
*  ポリモーフィズムによって，新しいクラスを追加する際，既存のコードを変更せずに機能を拡張できるため，システム全体に影響を与えにくい形で保守・拡張が行える

### 抽象化と保守性
*  オブジェクト指向では，クラスやオブジェクトに対して必要な機能のみを定義し，具体的な実装を隠すことができる（⇒ 抽象化）
*  コードをより抽象的なレベルで扱うことで，実装の変更が他のクラスやオブジェクトに直接影響を与えにくくなり，システム全体の理解や保守が容易になる

# SOLID原則の概要
*  有名なソフトウェア開発者のRobert C. Martinが2000年に発表 ⇒ [原著論文](https://web.archive.org/web/20150906155800/http:/www.objectmentor.com/resources/articles/Principles_and_Patterns.pdf)
*  オブジェクト指向プログラミングをする上で，重要となる5つの原則
>*  単一責任の原則（**S**ingle Responsibility Principle）
>*  開放・閉鎖の原則（**O**pen-Closed Principle）
>*  リスコフの置換原則（**L**iskov Substitution Principle）
>*  インタフェース分離の原則（**I**nterface Segregation Principle）
>*  依存関係逆転の原則（**D**ependency InversionPrinciple
*  SOLIDはそれらの頭文字をとったもの
*  修正性（変更のしやすさ）は，SOLID原則遵守による主な効果の一つ
*  SOLID原則の遵守によって修正性を向上させることで，保守性を向上する
>*  修正性が高くなると，依存関係がシンプルで，モジュール性（システムを独立した部品（モジュール）に分割する能力）が高くなる
>*  モジュール性が高くなると，再利用性と試験性（テストしやすさ）も高くなる
>*  修正しやすいコードは，解析性（理解しやすさ）も高い
*  カプセル化，継承，ポリモーフィズムを理解しただけでは，よい（変更のしやすい／保守性の高い）システムを設計することはできない
*  SOLID原則を理解することで，これらの要素（カプセル化，継承，ポリモーフィズム）を効果的に利用したよいシステムが設計できるようになる

# 単一責任の原則

## 単一責任の原則（Single Responsibility Principle, SRP）とは
*  オブジェクト指向では，クラス（オブジェクト）がそれぞれの責務（責任）を持ち，互いに協調することでシステムの機能を実現する

  
---
**単一責任の原則:**
*  クラスは一つの責任（役割）を持つ:
>*  一つのクラスは一つの目的や役割にフォーカスし，その役割に対してのみ責任を持つべき
>*  複数の目的を持たせると，変更が発生した場合にその影響が広範囲に及び、保守が困難になる
*  クラスは一つの変更理由を持つ:
>*  クラスに加えられる変更の理由は一つだけであるべき
>*  例えば，UI（ユーザインタフェース）の変更やデータベースの変更が同じクラスに影響を与えないように設計する
>*  一つのクラスが複数の責任を持っていると，一つの変更が別の部分に不具合をもたらす可能性がある
---

*  単一の責任を持つ ＝ 変更理由が一つ
*  責任（変更理由）が2つ以上あるクラス:
>*  クラスが役割・機能がわかりづらくなる
>*  クラスの名前が曖昧なものになる ⇒ コードがわかりづらくなる
>*  それぞれの責務がクラス内部で結合しやすくなる ⇒ 例えば，複数の責務（メソッド）で同じインスタンス属性を共有していると，あるメソッドの変更が他のメソッドにも影響を与える
>*  結合しやすくなることで，変更箇所がわかりづらくなる

<img src="./fig/SOLID_SRP.png" width="400">



## 例1

### 単一責任の原則に違反している例
*  以下のコードにおける`User`クラスは，ユーザ情報管理機能（役割1）とデータ保存機能（役割2）を持つクラスとして定義されている
*  これにより，クラスの変更理由が増え，保守性が低下する  

<img src="./fig/SOLID_SRP_example1.png" width="200">

In [ ]:
class User:
    # ユーザ情報管理機能
    def __init__(self, name: str) -> None:
        self.name = name

    # ユーザデータ保存機能
    def save_user_info(self, text: str) -> None:
        with open(f'{self.name}.txt', 'w') as file:
            file.write(f'Name: {self.name}\n{text}')

user = User('Alice')
user.save_user_info('alice@example.com')

# 保存したデータの内容表示
with open(f'{user.name}.txt', 'r') as f:
    txt = f.read()
    print(f'{user.name}.txtの内容:\n{txt}')

*  上のコードにおいて，会社情報管理機能を持つ`Company`クラスを新たに追加することを考える
*  このとき，会社情報を保存する機能（`User`クラスの役割2）を利用するためには，`Company`クラスの中でも同様の機能を定義する必要がある
*  これは，同じ機能を持つメソッドを2カ所で記述していることになるので，コードが冗長となり，可読性を低下させる

In [ ]:
class User:
    # ユーザ情報管理機能
    def __init__(self, name: str) -> None:
        self.name = name

    # ユーザデータ保存機能
    def save_user_info(self, text: str) -> None:
        with open(f'{self.name}.txt', 'w') as file:
            file.write(f'Name: {self.name}\n{text}')

class Company:
    # 会社情報管理機能
    def __init__(self, name: str) -> None:
        self.name = name

    # 会社データ保存機能
    def save_company_info(self, text: str) -> None:
        with open(f'{self.name}.txt', 'w') as file:
            file.write(f'Name: {self.name}\n{text}')

user = User('Alice')
user.save_user_info('alice@example.com')

company = Company('Example')
company.save_company_info('Kinuta, Setagaya-ku, Tokyo')

# 保存したデータの内容表示（ユーザ）
with open(f'{user.name}.txt', 'r') as f:
    txt = f.read()
    print(f'{user.name}.txtの内容:\n{txt}\n')

# 保存したデータの内容表示（会社）
with open(f'{company.name}.txt', 'r') as f:
    txt = f.read()
    print(f'{company.name}.txtの内容:\n{txt}\n')

### 単一責任の原則を遵守した例
*  上のコードにおける`User`クラスのユーザ情報管理機能とデータ保存機能は，単一責任の原則に従うと，それぞれの役割を持つクラスに分割すべきである
*  以下のコードでは，`User`クラスはユーザ情報管理機能のみを担い，データ保存機能は`FileManager`クラスが担う

<img src="./fig/SOLID_SRP_example2.png" width="400">

In [ ]:
class User:
    def __init__(self, name):
        self.name = name

class FileManager:
    def save_info(self, obj, text):
        with open(f'{obj.name}.txt', 'w') as file:
            file.write(f'Name: {obj.name}\n{text}')

user = User('Bob')
filemaneger = FileManager()
filemaneger.save_info(user, 'bob@example.com')

# 保存したデータの内容表示（ユーザ）
with open(f'{user.name}.txt', 'r') as f:
    txt = f.read()
    print(f'{user.name}.txtの内容:\n{txt}\n')

*  上記コードのように役割を分けておくと，会社情報管理機能を持つ`Company`クラスを追加した場合，`Company`クラスのインスタンスに対してもデータ保存機能（`FileManager`クラスの役割）を適用することができる
*  これにより，コードの冗長性がなくなり，可読性が向上する

In [ ]:
class User:
    def __init__(self, name):
        self.name = name

class Company:
    def __init__(self, name):
        self.name = name

class FileManager:
    def save_info(self, obj, text):
        with open(f'{obj.name}.txt', 'w') as file:
            file.write(f'Name: {obj.name}\n{text}')

user = User('Carol')
filemaneger = FileManager()
filemaneger.save_info(user, 'carol@example.com')

company = Company('Example')
filemaneger.save_info(company, 'Kinuta, Setagaya-ku, Tokyo')

# 保存したデータの内容表示（ユーザ）
with open(f'{user.name}.txt', 'r') as f:
    txt = f.read()
    print(f'{user.name}.txtの内容:\n{txt}\n')

# 保存したデータの内容表示（会社）
with open(f'{company.name}.txt', 'r') as f:
    txt = f.read()
    print(f'{company.name}.txtの内容:\n{txt}\n')

## 例2

### 単一責任の原則に違反している例
*  `User`クラス:
>*  ユーザの名前とメールアドレスをインスタンス属性`name`, `email`として持つ
*  `Authenticator`クラス:
>*  インスタンス属性`users`: 登録ユーザ（Userオブジェクト）を要素とするリスト
>*  `register_user`メソッド: `users`にUserオブジェクトを追加（ユーザの登録）
>*  `authenticate`メソッド: 仮引数`email`に受け取ったメールアドレスを持つユーザが`users`に含まれているかチェック（ユーザの認証）
>*  `logging`メソッド: 認証ログ（メールアドレスと認証結果）を auth_log.txt に書き込む
*  問題点: `Authenticator`クラスが，ユーザの登録，認証，ログの記録といった複数の機能を持つ ⇒ 単一責任の原則に違反

In [ ]:
# ユーザを表現するクラス
class User:
    def __init__(self, name: str, email: str) -> None:
        self.name = name
        self.email = email

# 認証とログ管理を行うクラス（単一責任の原則に違反）
class Authenticator:
    def __init__(self) -> None:
        self.users: list[User] = [] # 登録ユーザのリスト

    # ユーザの登録
    def register_user(self, user: User) -> None:
        self.users.append(user)
        print(f'{user.name}が登録されました。')

    # ユーザの認証
    def authenticate(self, email: str) -> bool:
        for user in self.users:
            if user.email == email:
                print(f'{user.name}が認証されました。')
                return True
        print('認証に失敗しました。')
        return False

    # ログの記録
    def logging(self, email: str, success: bool) -> None:
        with open('auth_log.txt', 'a') as log_file:
            if success:
                status = '成功'
            else:
                status = '失敗'
            log_file.write(f'{email}による認証: {status}\n')
        print('認証ログが記録されました。')


auth = Authenticator() # Authenticatorクラスのインスタンス生成

# Userクラスのインスタンス生成
usr1 = User('Alice', 'alice@example.com')
usr2 = User('Bob', 'bob@abc.com')

# ユーザ登録
auth.register_user(usr1) # output: Aliceが登録されました。
auth.register_user(usr2) # output: Bobが登録されました。

# ユーザ認証とログの記録
email1 = 'bob@abc.com'
log1 = auth.authenticate('bob@abc.com') # output: Bobが認証されました。
auth.logging(email1, log1) # output: 認証ログが記録されました。
email2 = 'eve@abc.com'
log2 = auth.authenticate('eve@abc.com') # output: 認証に失敗しました。
auth.logging(email2, log2) # output: 認証ログが記録されました。

# 認証ログファイルの内容表示
with open('auth_log.txt', 'r') as f:
    txt = f.read()
    print(f'認証ログファイル「auth_log.txt」の内容:\n{txt}\n')

### 単一責任の原則を遵守した例
*  上のコードにおいて，`Authenticator`クラスが持つ役割のユーザの登録，認証，ログの記録を以下の3つのクラスに分けて定義する
>*  `UserManeger`クラス: ユーザの登録と管理を行う
>*  `UserAuthenticator`クラス: ユーザの認証を行う
>*  `AuthLogger`クラス: 認証ログを管理する


In [ ]:
# ユーザを表現するクラス
class User:
    def __init__(self, name: str, email: str) -> None:
        self.name = name
        self.email = email

# ユーザの登録と管理を担当するクラス
class UserManeger:
    # 登録ユーザの管理
    def __init__(self) -> None:
        self.users: list[User] = []

    # ユーザの登録
    def register_user(self, user: User) -> None:
        self.users.append(user)
        print(f'{user.name}が登録されました。')

# ユーザの認証を担当するクラス
class UserAuthenticator:
    def authenticate(self, email: str, manager: UserManeger) -> bool:
        for user in manager.users:
            if user.email == email:
                print(f'{user.name}が認証されました。')
                return True
        print('認証に失敗しました。')
        return False

# 認証ログを管理するクラス
class AuthLogger:
    def logging(self, email: str, success: bool) -> None:
        with open('auth_log2.txt', 'a') as log_file:
            if success:
                status = '成功'
            else:
                status = '失敗'
            log_file.write(f'{email}による認証: {status}\n')
        print('認証ログが記録されました。')

# Userクラスのインスタンス生成
usr1 = User('Alice', 'alice@example.com')
usr2 = User('Bob', 'bob@abc.com')

# 各クラスからインスタンスを生成
manager = UserManeger()
authenticator = UserAuthenticator()
logger = AuthLogger()

# ユーザ登録
manager.register_user(usr1) # output: Aliceが登録されました。
manager.register_user(usr2) # output: Bobが登録されました。

# ユーザ認証とログの記録
email1 = 'bob@abc.com'
log1 = authenticator.authenticate('bob@abc.com', manager) # output: Bobが認証されました。
logger.logging(email1, log1) # output: 認証ログが記録されました。
email2 = 'eve@abc.com'
log2 = authenticator.authenticate('eve@abc.com', manager) # output: 認証に失敗しました。
logger.logging(email2, log2) # output: 認証ログが記録されました。

# 認証ログファイルの内容表示
with open('auth_log2.txt', 'r') as f:
    txt = f.read()
    print(f'認証ログファイル「auth_log.txt」の内容:\n{txt}\n')

## 単一責任の原則の利点
*  保守性の向上: 単一の責任を持つクラスは，変更が必要な場合に特定しやすく，修正も容易
*  試験性の向上: 責任が明確なクラスは，テストの作成と実行が容易になる
*  再利用性の向上: 単一の責任を持つクラスは，他のシステムでも再利用しやすくなる
*  可読性の向上: 各クラスの目的が明確になり，コード全体の理解が容易になる

# 参考資料
*  Robert C.Martin (著), 角征典 (翻訳), 高木正弘 (翻訳), [Clean Architecture 達人に学ぶソフトウェアの構造と設計](https://www.kadokawa.co.jp/product/301806000678/), KADOKAWA, 2018.
*  伊藤裕一, [速習 Python 3 中: オブジェクト指向編 Kindle版](https://www.amazon.co.jp/%E9%80%9F%E7%BF%92-Python-3-%E4%B8%AD-%E3%82%AA%E3%83%96%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E6%8C%87%E5%90%91%E7%B7%A8-ebook/dp/B01N04UBYI), 2016
*  ひらまつしょうたろう, [Python でわかる オブジェクト指向 とはなにか？【Python オブジェクト指向 の「なぜ？」を「徹底的に」解説】](https://www.udemy.com/course/oop-python/?couponCode=KEEPLEARNING), Udemy, 最終更新日 2023/7.
*  ひらまつしょうたろう, [Python で身につける オブジェクト指向【SOLID原則+デザインパターンで、オブジェクト指向設計 の基礎を習得！】](https://www.udemy.com/course/python-solid-design-pattern/?couponCode=KEEPLEARNING), Udemy, 最終更新日 2024/6.
<!-- *  河合昭男, [ゼロからわかる UML超入門](https://gihyo.jp/book/2017/978-4-7741-9005-1), 技術評論社, 2010. -->
<!-- *  竹政昭利, 林田幸司, 大西洋平, 三村次朗, 藤本陽啓, 伊藤宏幸, [かんたんUML入門 ［改訂2版］ Kindle版](https://gihyo.jp/book/2017/978-4-7741-9039-6), 技術評論社, 2017. -->
<!-- *  Bill Lubanovic (著), 鈴木駿 (監訳), 長尾高弘 (訳), [入門 Python 3 第2版](https://www.oreilly.co.jp/books/9784873119328/), オライリージャパン, 2021. -->
<!-- *  Guido van Rossum (著), 鴨澤眞夫 (翻訳), [Pythonチュートリアル 第4版](https://www.oreilly.co.jp/books/9784873119359/), オライリージャパン, 2021. -->

